# Empirical Analysis of Sorting Algorithms

## **0. Importing Required Libraries**

In [1]:
#

## **1. Introduction**

## **2. Objective**

## **3 Implementing Sorting Algorithms**

### **3.1 Quick Sort**

In [2]:
def quick_sort(arr):
    if len(arr) <= 1:
        return arr

    pivot = arr[len(arr) // 2]

    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]

    return quick_sort(left) + middle + quick_sort(right)

### **3.2 Merge Sort**

In [3]:
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

### **3.3 Heap Sort**

In [4]:
def heapify(arr, n, i):
    largest = i
    l = 2 * i + 1
    r = 2 * i + 2
    if l < n and arr[l] > arr[largest]:
        largest = l
    if r < n and arr[r] > arr[largest]:
        largest = r
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest)

def heap_sort(arr):
    n = len(arr)
    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)
    for i in range(n - 1, 0, -1):
        arr[0], arr[i] = arr[i], arr[0]
        heapify(arr, i, 0)


### **3.4 Insertion Sort**

In [5]:
#

## **4. Experimental Setup**

In [6]:
test_arrays = [
    [],
    [1],
    [5, 3, 1, 4, 2],
    [1, 2, 3, 4, 5],
    [5, 4, 3, 2, 1],
    [2, 2, 2, 2, 2],
    [3, 1, 4, 1, 5, 9, 2],
]

In [7]:
for arr in test_arrays:
    print(f"Original: {arr}")
    print(f"Quick Sort: {quick_sort(arr)}")
    print(f"Merge Sort: {merge_sort(arr)}")
    print(f"Heap Sort: {heap_sort(arr)}")
    print("--------------------------------")

Original: []
Quick Sort: []
Merge Sort: []
Heap Sort: None
--------------------------------
Original: [1]
Quick Sort: [1]
Merge Sort: [1]
Heap Sort: None
--------------------------------
Original: [5, 3, 1, 4, 2]
Quick Sort: [1, 2, 3, 4, 5]
Merge Sort: [1, 2, 3, 4, 5]
Heap Sort: None
--------------------------------
Original: [1, 2, 3, 4, 5]
Quick Sort: [1, 2, 3, 4, 5]
Merge Sort: [1, 2, 3, 4, 5]
Heap Sort: None
--------------------------------
Original: [5, 4, 3, 2, 1]
Quick Sort: [1, 2, 3, 4, 5]
Merge Sort: [1, 2, 3, 4, 5]
Heap Sort: None
--------------------------------
Original: [2, 2, 2, 2, 2]
Quick Sort: [2, 2, 2, 2, 2]
Merge Sort: [2, 2, 2, 2, 2]
Heap Sort: None
--------------------------------
Original: [3, 1, 4, 1, 5, 9, 2]
Quick Sort: [1, 1, 2, 3, 4, 5, 9]
Merge Sort: [1, 1, 2, 3, 4, 5, 9]
Heap Sort: None
--------------------------------


## **5. Results and Analysis**

## **6. Conclusion**

### **6.1 Key Findings**

### **6.2 Choosing the Optimal Sorting Algorithm**

### **6.3 Final Remarks**